In [ ]:
import os
import numpy as np_utils
from PIL import Image
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
#from keras.layers import LeakyReLU
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
#from keras.models import Model
import sys
import tensorflow as tf
import pathlib
from shutil import copy
import tensorflow_io as tfio 
import tensorflow_addons as tfa

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def preprocess1(path, label):
    # read image
    image_raw = tf.io.read_file(path['raw_input']) 
    image_sin = tf.io.read_file(path['sin_input'])
    #decode 
    #image_sin = tf.image.decode_jpeg(image_sin, channels=3)
    image_raw=tfio.experimental.image.decode_tiff(image_raw)
    image_sin = tfio.experimental.image.decode_tiff(image_sin)
    # rsize o the same input size of resnet 
    image_raw = tf.image.resize(image_raw, [112, 112]) 
    image_sin = tf.image.resize(image_sin, [112, 112])
    image_raw= image_raw[:,:,0:3]
    image_sin= image_sin[:,:,0:3]
     # normalize
    image_raw /= 255.0 
    image_sin /= 255.0 
    return {'raw_input':image_raw,'sin_input':image_sin}, label

In [ ]:
def preprocess2(path, label):
    # read image
    image_raw = tf.io.read_file(path['raw_input']) 
    image_sin = tf.io.read_file(path['sin_input'])
    #decode 
    image_sin = tf.image.decode_jpeg(image_sin, channels=3)
    image_raw=tfio.experimental.image.decode_tiff(image_raw)
    #image_sin = tfio.experimental.image.decode_tiff(image_sin)
    # rsize o the same input size of resnet 
    image_raw = tf.image.resize(image_raw, [112, 112]) 
    image_sin = tf.image.resize(image_sin, [112, 112])
    image_raw= image_raw[:,:,0:3]
    image_sin= image_sin[:,:,0:3]
     # normalize
    image_raw /= 255.0 
    image_sin /= 255.0 
    return {'raw_input':image_raw,'sin_input':image_sin}, label

In [ ]:
def preprocess10(path, label):
    # read image
    image_raw = tf.io.read_file(path['raw_input']) 
    image_sin = tf.io.read_file(path['sin_input'])
    #decode 
    image_sin = tf.image.decode_jpeg(image_sin, channels=3)
    image_raw=tf.image.decode_jpeg(image_raw)
    # rsize o the same input size of resnet 
    image_raw = tf.image.resize(image_raw, [112, 112]) 
    image_sin = tf.image.resize(image_sin, [112, 112])
    image_raw= image_raw[:,:,0:3]
    image_sin= image_sin[:,:,0:3]
     # normalize
    image_raw /= 255.0 
    image_sin /= 255.0 
    return {'raw_input':image_raw,'sin_input':image_sin}, label

In [ ]:
def get_dataset(raw,sin,target,number):
    #get the path to dataset
    raw_path_r = pathlib.Path(raw)
    #get the last two level of the whole path , like damage/111.jpeg or no_damage/11111.jpeg
    raw_path = list(raw_path_r.glob('*/*')) 
    #transform all path to string, preparing for reading 
    raw_path = [str(path) for path in raw_path]
        #get the path to dataset
    sin_path = pathlib.Path(sin)
    #get the last two level of the whole path , like damage/111.jpeg or no_damage/11111.jpeg
    sin_path = list(sin_path.glob('*/*')) 
    #transform all path to string, preparing for reading 
    sin_path = [str(path) for path in sin_path]
        #get the label, damage or no_damage
    label_names_train = sorted(item.name for item in raw_path_r.glob('*/') if item.is_dir())
    #transform damage to 0, no_damage to 1
    all_image_labels_train = [label_to_index[pathlib.Path(path).parent.name] for path in raw_path]
    #get the data path and label
    ds_train = tf.data.Dataset.from_tensor_slices(({'raw_input':raw_path,'sin_input':sin_path}, all_image_labels_train))
    #preprocess fuciton, transform the data path to data and other preprocess
    if (number==1):
        ds_train=ds_train.map(preprocess2)
    else:
        ds_train=ds_train.map(preprocess1)
    #else:
    #ds_train=ds_train.map(preprocess10)
    return ds_train

In [ ]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x, shape=(-1, patch_num_y, window_size, patch_num_x, window_size, channels)
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape=(-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        shape=(-1, patch_num_y, patch_num_x, window_size, window_size, channels),
    )
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape=(-1, height, width, channels))
    return x


class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super(DropPath, self).__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x):
        input_shape = tf.shape(x)
        batch_size = input_shape[0]
        rank = x.shape.rank
        shape = (batch_size,) + (1,) * (rank - 1)
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape, dtype=x.dtype)
        path_mask = tf.floor(random_tensor)
        output = tf.math.divide(x, 1 - self.drop_prob) * path_mask
        return output

In [ ]:
class WindowAttention(layers.Layer):
    def __init__(
        self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs
    ):
        super(WindowAttention, self).__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        num_window_elements = (2 * self.window_size[0] - 1) * (
            2 * self.window_size[1] - 1
        )
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=tf.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        self.relative_position_index = tf.Variable(
            initial_value=tf.convert_to_tensor(relative_position_index), trainable=False
        )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm=(0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(
            self.relative_position_index, shape=(-1,)
        )
        relative_position_bias = tf.gather(
            self.relative_position_bias_table, relative_position_index_flat
        )
        relative_position_bias = tf.reshape(
            relative_position_bias, shape=(num_window_elements, num_window_elements, -1)
        )
        relative_position_bias = tf.transpose(relative_position_bias, perm=(2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32
            )
            attn = (
                tf.reshape(attn, shape=(-1, nW, self.num_heads, size, size))
                + mask_float
            )
            attn = tf.reshape(attn, shape=(-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv

In [ ]:
class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super(SwinTransformer, self).__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, shape=[-1, self.window_size * self.window_size]
            )
            attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(
                mask_windows, axis=2
            )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(initial_value=attn_mask, trainable=False)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape=(-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2]
            )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, shape=(-1, self.window_size * self.window_size, channels)
        )
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows, shape=(-1, self.window_size, self.window_size, channels)
        )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
        )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2]
            )
        else:
            x = shifted_x

        x = tf.reshape(x, shape=(-1, height * width, channels))
        x = self.drop_path(x)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(PatchExtract, self).__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[0]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super(PatchEmbedding, self).__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch, delta=1)
        return self.proj(patch) + self.pos_embed(pos)


class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super(PatchMerging, self).__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape=(-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

In [ ]:
inputs_raw = tf.keras.Input(shape=(112, 112,3),name='raw_input')

#GlobalAveragePooling1D = tf.keras.layers.GlobalAveragePooling1D() 
x1 = PatchExtract(patch_size)(inputs_raw)
x1 = PatchEmbedding(num_patch_x * num_patch_x, embed_dim)(x1)
x1 = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x,num_patch_x),
    num_heads=8,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_x),
    num_heads=8,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = PatchMerging((56, 56), embed_dim=64)(x1)
x1 = SwinTransformer(
    dim=embed_dim*2,
    num_patch=(28, 28),
    num_heads=8,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = SwinTransformer(
    dim=embed_dim*2,
    num_patch=(28, 28),
    num_heads=8,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = PatchMerging((28, 28), embed_dim=128)(x1)
x1 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
x1 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x1)
#add0 = tf.keras.layers.add([x1,x1_r])
x1 = PatchMerging((14, 14), embed_dim=embed_dim*4)(x1)

#x1= tf.keras.layers.GlobalAveragePooling1D()(x1)

inputs_sin = tf.keras.Input(shape=(112, 112,3),name='sin_input')
x2 = PatchExtract((2,2))(inputs_sin)
x2 = PatchEmbedding(num_patch_x * num_patch_x,64)(x2)
x2 = SwinTransformer(
    dim=embed_dim,
    num_patch=(56, 56),
    num_heads=4,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = SwinTransformer(
    dim=embed_dim,
    num_patch=(56, 56),
    num_heads=4,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = PatchMerging((56, 56), embed_dim=64)(x2)
x2 = SwinTransformer(
    dim=embed_dim*2,
    num_patch=(28, 28),
    num_heads=8,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = SwinTransformer(
    dim=embed_dim*2,
    num_patch=(28, 28),
    num_heads=8,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = PatchMerging((28, 28), embed_dim=128)(x2)
x2 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=0,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = SwinTransformer(
    dim=embed_dim*4,
    num_patch=(14, 14),
    num_heads=16,
    window_size=7,
    shift_size=4,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
    num_mlp=1024
)(x2)
x2 = PatchMerging((14, 14), embed_dim=embed_dim*4)(x2)
#x2 = PatchMerging((28, 28), embed_dim=64*2)(x2)


#x2= tf.keras.layers.GlobalAveragePooling1D()(x2)


concat = tf.keras.layers.concatenate([x2,x1])
x=tf.keras.layers.GlobalAveragePooling1D()(concat)
x = tf.keras.layers.Dropout(0.5)(x)
x=tf.keras.layers.Dense(1024,activation='gelu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x=tf.keras.layers.Dense(1024,activation='gelu')(x)
outputs=tf.keras.layers.Dense(20,activation='softmax')(x)
model = tf.keras.Model(inputs=[inputs_raw,inputs_sin], outputs=[outputs])#compile the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [ ]:
%%time
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=40,restore_best_weights=True)
history=model.fit(ds_train.batch(32).shuffle(3500).prefetch(buffer_size=AUTOTUNE), epochs =150,verbose=1, validation_data=ds_t_v.batch(32).prefetch(buffer_size=AUTOTUNE),callbacks=[callback])